1. Instalación y Configuración Inicial


In [ ]:
# Instalamos PyCaret y otras bibliotecas necesarias
#!pip install pycaret matplotlib seaborn pandas numpy

In [ ]:
# Importamos las bibliotecas necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pycaret.classification import *
import warnings
warnings.filterwarnings('ignore')

# Configuración para visualizaciones
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 8)
sns.set(style="whitegrid")

2. Carga y Exploración de Datos


In [ ]:
# Cargamos el dataset
df = pd.read_csv('data_set_integrado_modelo_final.csv')

# Verificamos las dimensiones del dataset
print(f"Dimensiones del dataset: {df.shape}")
print(f"Número de filas: {df.shape[0]}")
print(f"Número de columnas: {df.shape[1]}")

In [ ]:
# Exploramos las primeras filas del dataset
df.head()

In [ ]:
# Verificamos los tipos de datos
df.dtypes

In [ ]:
# Verificamos la distribución de las clases objetivo
print("Distribución de la variable 'label':")
label_counts = df['label'].value_counts()
print(label_counts)

# Visualizamos la distribución
plt.figure(figsize=(10, 6))
sns.countplot(x='label', data=df)
plt.title('Distribución de Clases en el Dataset Completo')
plt.ylabel('Conteo')
plt.xlabel('Clase')
plt.show()

In [ ]:
# Verificamos las empresas únicas en el dataset
empresas = df['id_empresa'].unique()
print(f"Número de empresas distintas: {len(empresas)}")
print("IDs de empresas:", empresas)

# Visualizamos la distribución de registros por empresa
plt.figure(figsize=(14, 8))
sns.countplot(x='id_empresa', data=df)
plt.title('Cantidad de Registros por Empresa')
plt.ylabel('Conteo')
plt.xlabel('ID de Empresa')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Examinamos la distribución de clases por empresa
plt.figure(figsize=(16, 10))
for i, emp in enumerate(empresas, 1):
    plt.subplot(2, (len(empresas)+1)//2, i)
    emp_df = df[df['id_empresa'] == emp]
    sns.countplot(x='label', data=emp_df)
    plt.title(f'Empresa {emp}')
    plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

3. Preparación de Datos


In [ ]:
# Limpieza básica del dataset
def preparar_datos(dataframe):
    # Hacemos una copia para no modificar el original
    df_clean = dataframe.copy()
    
    # 1. Eliminar filas sin etiqueta
    df_clean = df_clean.dropna(subset=['label'])
    
    # 2. Verificar valores nulos
    print("Valores nulos por columna:")
    print(df_clean.isnull().sum())
    
    # 3. Eliminar columnas con más del 50% de valores faltantes
    threshold = 0.5
    missing_ratio = df_clean.isnull().mean()
    cols_to_drop = missing_ratio[missing_ratio > threshold].index.tolist()
    if cols_to_drop:
        print(f"Columnas eliminadas por tener más del 50% de valores nulos: {cols_to_drop}")
        df_clean = df_clean.drop(columns=cols_to_drop)
    
    # 4. Eliminar cualquier fila restante con valores faltantes
    rows_before = df_clean.shape[0]
    df_clean = df_clean.dropna()
    rows_after = df_clean.shape[0]
    print(f"Se eliminaron {rows_before - rows_after} filas con valores nulos.")
    
    return df_clean

# Aplicamos la limpieza
df_clean = preparar_datos(df)

# Verificamos la forma del dataset después de la limpieza
print(f"Dimensiones después de la limpieza: {df_clean.shape}")

4. Análisis Exploratorio de Características


In [ ]:
# Analizamos las correlaciones entre variables numéricas
# Seleccionamos solo columnas numéricas
numeric_cols = df_clean.select_dtypes(include=['int64', 'float64']).columns.tolist()
numeric_cols = [col for col in numeric_cols if col != 'id_empresa' and col != 'id_cotizacion']

# Calculamos la matriz de correlación
corr_matrix = df_clean[numeric_cols].corr()

# Visualizamos el mapa de calor de correlaciones
plt.figure(figsize=(16, 14))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=False, cmap='coolwarm', center=0, linewidths=0.5)
plt.title('Matriz de Correlación de Variables Numéricas')
plt.tight_layout()
plt.show()

In [ ]:
# Identificamos variables altamente correlacionadas para posible eliminación
high_corr_threshold = 0.9
high_corr_pairs = []

for i in range(len(numeric_cols)):
    for j in range(i+1, len(numeric_cols)):
        corr = abs(corr_matrix.iloc[i, j])
        if corr > high_corr_threshold:
            high_corr_pairs.append((numeric_cols[i], numeric_cols[j], corr))

# Mostramos las variables altamente correlacionadas
if high_corr_pairs:
    print("Variables altamente correlacionadas (|r| > 0.9):")
    for var1, var2, corr in high_corr_pairs:
        print(f"{var1} - {var2}: {corr:.4f}")
else:
    print("No se encontraron pares de variables con correlación mayor a 0.9")

In [ ]:
# Función para construir y evaluar modelos para una empresa específica
def modelar_empresa(df, empresa_id):
    print(f"\n{'='*50}")
    print(f"Modelado para la empresa {empresa_id}")
    print(f"{'='*50}")
    
    # Filtramos solo los datos de esta empresa
    empresa_df = df[df['id_empresa'] == empresa_id].copy()
    print(f"Número de registros para la empresa {empresa_id}: {empresa_df.shape[0]}")
    
    # Verificamos la distribución de clases
    print("\nDistribución de clases:")
    print(empresa_df['label'].value_counts())
    
    # Verificamos si hay suficientes datos
    if empresa_df.shape[0] < 30:
        print(f"ADVERTENCIA: La empresa {empresa_id} tiene muy pocos registros para modelar adecuadamente.")
        return None, None, None
    
    # Eliminamos columnas no necesarias para el modelado
    cols_to_drop = ['id_cotizacion', 'id_empresa']
    modelo_df = empresa_df.drop(columns=cols_to_drop)
    
    # Inicializamos PyCaret
    exp_name = f"Empresa_{empresa_id}"
    print("\nConfigurando ambiente PyCaret...")
    
    clf = setup(
        data=modelo_df,
        target='label',
        session_id=42,
        experiment_name=exp_name,
        normalize=True,
        transformation=True,
        ignore_features=['fecha', 'nombre'],
        html=False,
        verbose=True
    )
    
    # Comparamos modelos
    print("\nComparando modelos...")
    top_models = compare_models(n_select=3)
    
    # Si top_models es un solo modelo y no una lista, lo convertimos a lista
    if not isinstance(top_models, list):
        top_models = [top_models]
    
    # Creamos un diccionario para almacenar los resultados
    resultados = {}
    
    # Evaluamos cada uno de los modelos top
    print("\nEvaluando modelos seleccionados...")
    for i, model in enumerate(top_models):
        model_name = str(type(model).__name__)
        print(f"\nEvaluando {model_name}...")
        
        # Evaluamos el modelo
        evaluation = evaluate_model(model)
        
        # Creamos un modelo ajustado (tuneado)
        print(f"Tuneando {model_name}...")
        tuned_model = tune_model(model)
        
        # Visualizamos la matriz de confusión del modelo ajustado
        plot_model(tuned_model, plot='confusion_matrix', save=True)
        
        # Visualizamos la curva AUC del modelo ajustado (para multiclase)
        plot_model(tuned_model, plot='auc', save=True)
        
        # Visualizamos feature importance
        if hasattr(tuned_model, 'feature_importances_') or hasattr(tuned_model, 'coef_'):
            plot_model(tuned_model, plot='feature', save=True)
        
        # Guardamos el modelo
        model_path = f"modelo_{empresa_id}_{model_name}"
        print(f"Guardando modelo en {model_path}...")
        save_model(tuned_model, model_path)
        
        resultados[model_name] = tuned_model
    
    # Finalizamos
    print(f"\nModelado completo para la empresa {empresa_id}")
    
    # Retornamos el DataFrame de la empresa, los modelos y los resultados
    return empresa_df, top_models, resultados

In [ ]:
# Ejecutamos el modelado para cada empresa
resultados_por_empresa = {}

for empresa_id in empresas:
    print(f"\nProcesando empresa {empresa_id}...")
    empresa_df, modelos, resultados = modelar_empresa(df_clean, empresa_id)
    
    if resultados:
        resultados_por_empresa[empresa_id] = {
            'dataframe': empresa_df,
            'modelos': modelos,
            'resultados': resultados
        }

6. Análisis Comparativo de Modelos entre Empresas


In [ ]:
# Creamos un DataFrame con los resultados comparativos
resumen_empresas = pd.DataFrame(columns=['Empresa', 'Mejor_Modelo', 'Accuracy', 'Precision', 'Recall', 'F1', 'Num_Registros'])

for empresa_id, datos in resultados_por_empresa.items():
    if 'modelos' in datos and datos['modelos']:
        # Obtenemos el mejor modelo (el primero de la lista)
        mejor_modelo = datos['modelos'][0]
        modelo_nombre = str(type(mejor_modelo).__name__)
        
        # Obtenemos métricas del pull_metrics 
        metricas = pull()
        mejor_modelo_metricas = metricas[metricas['Model'] == modelo_nombre].iloc[0]
        
        # Añadimos al DataFrame
        resumen_empresas = resumen_empresas.append({
            'Empresa': empresa_id,
            'Mejor_Modelo': modelo_nombre,
            'Accuracy': mejor_modelo_metricas['Accuracy'],
            'Precision': mejor_modelo_metricas['Prec. Macro'],
            'Recall': mejor_modelo_metricas['Recall Macro'],
            'F1': mejor_modelo_metricas['F1 Macro'],
            'Num_Registros': datos['dataframe'].shape[0]
        }, ignore_index=True)

# Mostramos el resumen
print("Resumen Comparativo de Modelos por Empresa:")
resumen_empresas

In [ ]:
# Visualizamos las métricas por empresa
plt.figure(figsize=(14, 10))

# Accuracy
plt.subplot(2, 2, 1)
sns.barplot(x='Empresa', y='Accuracy', data=resumen_empresas)
plt.title('Accuracy por Empresa')
plt.xticks(rotation=45)

# Precision
plt.subplot(2, 2, 2)
sns.barplot(x='Empresa', y='Precision', data=resumen_empresas)
plt.title('Precision por Empresa')
plt.xticks(rotation=45)

# Recall
plt.subplot(2, 2, 3)
sns.barplot(x='Empresa', y='Recall', data=resumen_empresas)
plt.title('Recall por Empresa')
plt.xticks(rotation=45)

# F1
plt.subplot(2, 2, 4)
sns.barplot(x='Empresa', y='F1', data=resumen_empresas)
plt.title('F1-Score por Empresa')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Gráfico de comparación de modelos por empresa
plt.figure(figsize=(12, 8))
sns.countplot(x='Mejor_Modelo', data=resumen_empresas)
plt.title('Distribución de Mejores Modelos por Empresa')
plt.xticks(rotation=45)
plt.ylabel('Conteo')
plt.show()

7. Análisis de Importancia de Características


In [ ]:
# Función para analizar las características más importantes para cada empresa
def analizar_features_importancia():
    for empresa_id, datos in resultados_por_empresa.items():
        if 'resultados' in datos and datos['resultados']:
            print(f"\n{'='*50}")
            print(f"Características importantes para la empresa {empresa_id}")
            print(f"{'='*50}")
            
            # Iteramos por los modelos disponibles
            for modelo_nombre, modelo in datos['resultados'].items():
                print(f"\nModelo: {modelo_nombre}")
                
                # Intentamos obtener importancia de características si está disponible
                try:
                    # Para modelos basados en árboles
                    if hasattr(modelo, 'feature_importances_'):
                        # Obtenemos los nombres de las características del setup de PyCaret
                        feature_names = get_config('X_train').columns.tolist()
                        importances = modelo.feature_importances_
                        
                        # Crear DataFrame de importancias
                        feature_importance = pd.DataFrame({
                            'Feature': feature_names,
                            'Importance': importances
                        })
                        
                        # Ordenar por importancia
                        feature_importance = feature_importance.sort_values('Importance', ascending=False).reset_index(drop=True)
                        
                        # Mostrar las 10 características más importantes
                        print("Top 10 características más importantes:")
                        print(feature_importance.head(10))
                        
                        # Visualizar
                        plt.figure(figsize=(12, 8))
                        sns.barplot(x='Importance', y='Feature', data=feature_importance.head(15))
                        plt.title(f'Importancia de Características - Empresa {empresa_id} - {modelo_nombre}')
                        plt.tight_layout()
                        plt.show()
                    
                    # Para modelos lineales
                    elif hasattr(modelo, 'coef_'):
                        feature_names = get_config('X_train').columns.tolist()
                        
                        # Los coeficientes pueden tener diferentes formas según el modelo
                        if len(modelo.coef_.shape) == 1:
                            coefs = modelo.coef_
                        else:
                            # Promediamos coeficientes para todas las clases
                            coefs = np.mean(np.abs(modelo.coef_), axis=0)
                        
                        # Crear DataFrame de coeficientes
                        feature_importance = pd.DataFrame({
                            'Feature': feature_names,
                            'Coefficient': coefs
                        })
                        
                        # Ordenar por valor absoluto de coeficientes
                        feature_importance['AbsCoef'] = np.abs(feature_importance['Coefficient'])
                        feature_importance = feature_importance.sort_values('AbsCoef', ascending=False).reset_index(drop=True)
                        
                        # Mostrar las 10 características más importantes
                        print("Top 10 características más importantes:")
                        print(feature_importance[['Feature', 'Coefficient']].head(10))
                        
                        # Visualizar
                        plt.figure(figsize=(12, 8))
                        sns.barplot(x='AbsCoef', y='Feature', data=feature_importance.head(15))
                        plt.title(f'Importancia de Características - Empresa {empresa_id} - {modelo_nombre}')
                        plt.tight_layout()
                        plt.show()
                    
                    else:
                        print("Este modelo no proporciona medidas directas de importancia de características.")
                
                except Exception as e:
                    print(f"Error al analizar importancia de características: {str(e)}")

# Ejecutamos el análisis de características
analizar_features_importancia()

8. Prueba y Predicción con los Modelos Guardados


In [ ]:
# Función para cargar un modelo y hacer predicciones de ejemplo
def probar_modelo(empresa_id, modelo_nombre):
    try:
        # Cargamos el modelo
        model_path = f"modelo_{empresa_id}_{modelo_nombre}"
        modelo_cargado = load_model(model_path)
        
        print(f"Modelo cargado correctamente: {model_path}")
        
        # Obtenemos algunos datos de ejemplo para esta empresa
        empresa_df = resultados_por_empresa[empresa_id]['dataframe']
        
        # Tomamos las últimas 5 filas como ejemplo
        datos_ejemplo = empresa_df.tail(5).drop(columns=['label'])
        
        # Hacemos predicciones
        predicciones = predict_model(modelo_cargado, data=datos_ejemplo)
        
        print("\nPredicciones de ejemplo:")
        print(predicciones[['fecha', 'prediction_label', 'prediction_score']])
        
        return predicciones
    
    except Exception as e:
        print(f"Error al cargar o usar el modelo: {str(e)}")
        return None

# Probamos con un ejemplo para la primera empresa y su mejor modelo
if resultados_por_empresa:
    primera_empresa = list(resultados_por_empresa.keys())[0]
    mejor_modelo_nombre = str(type(resultados_por_empresa[primera_empresa]['modelos'][0]).__name__)
    
    print(f"Probando predicciones para la empresa {primera_empresa} con el modelo {mejor_modelo_nombre}")
    predicciones_ejemplo = probar_modelo(primera_empresa, mejor_modelo_nombre)

9. Conclusiones


In [ ]:
# Resumen y conclusiones
print("RESUMEN DEL ANÁLISIS:")
print("="*80)
print(f"Total de empresas analizadas: {len(resultados_por_empresa)}")

# Modelos más efectivos
if not resumen_empresas.empty:
    mejor_empresa = resumen_empresas.iloc[resumen_empresas['Accuracy'].idxmax()]
    print(f"\nEmpresa con mejor rendimiento: {mejor_empresa['Empresa']}")
    print(f"- Mejor modelo: {mejor_empresa['Mejor_Modelo']}")
    print(f"- Accuracy: {mejor_empresa['Accuracy']:.4f}")
    print(f"- F1-Score: {mejor_empresa['F1']:.4f}")
    
    # Modelos más frecuentes
    modelo_mas_comun = resumen_empresas['Mejor_Modelo'].mode()[0]
    print(f"\nModelo más común entre todas las empresas: {modelo_mas_comun}")
    
    # Promedio de métricas
    print("\nRendimiento promedio de los modelos:")
    print(f"- Accuracy promedio: {resumen_empresas['Accuracy'].mean():.4f}")
    print(f"- Precision promedio: {resumen_empresas['Precision'].mean():.4f}")
    print(f"- Recall promedio: {resumen_empresas['Recall'].mean():.4f}")
    print(f"- F1-Score promedio: {resumen_empresas['F1'].mean():.4f}")

print("\nCONCLUSIONES:")
print("="*80)
print("""
1. Hemos construido modelos de clasificación multiclase para predecir los movimientos 
   del mercado (SUBE, BAJA, MANTIENE) para cada empresa en el dataset.

2. Para cada empresa, hemos identificado las características más relevantes que 
   influyen en los movimientos de sus acciones.

3. Se han guardado los modelos entrenados para cada empresa, que pueden ser utilizados 
   para hacer predicciones futuras sobre nuevos datos.

4. Los resultados muestran diferentes niveles de predictibilidad entre las empresas, 
   lo que sugiere que algunos movimientos de mercado son más fáciles de predecir que otros.

5. Recomendaciones para mejorar los modelos:
   - Incorporar más datos históricos
   - Explorar técnicas de series temporales más avanzadas
   - Considerar variables macroeconómicas y noticias del sector
   - Implementar técnicas de balanceo de clases para mejorar la predicción de clases minoritarias
""")